In [2]:
# https://www.kaggle.com/c/digit-recognizer/submissions

import mxnet as mx
import pandas as pd
import numpy as np
import logging
from sklearn.model_selection import train_test_split

logging.getLogger().setLevel(logging.DEBUG)  # logging to stdout
path = './data/'

/home/huang/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [3]:
# Fix the seed
mx.random.seed(7)

# Set the compute context, GPU is available otherwise CPU
ctx = mx.gpu() if mx.test_utils.list_gpus() else mx.cpu()
print(ctx)

cpu(0)


In [11]:
print("model extract train init")
df_train = pd.read_csv(path + "train.csv")   # 42000 * 784
# y = (np.array(df_train['label'].values.tolist()).astype(np.int)).copy()
y = (np.array(df_train['label']))
# print(y)
df_train = df_train.drop(columns=['label'])
X = (np.array(df_train.values.tolist()).astype(np.float)).copy()
# print(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print("model extract train end")

model extract train init
model extract train end


In [13]:
print("model extrct test init")
df_test_final = pd.read_csv(path + "test.csv")
X_test_final = np.array(df_test_final.values.tolist()).astype(np.float)
print("model extract train end")

model extrct test init
model extract train end


In [15]:
batch_size = 100
train_iter = mx.io.NDArrayIter(X_train, y_train, batch_size, shuffle=True)
val_iter = mx.io.NDArrayIter(X_test, y_test, batch_size)

In [16]:
# Training 
# define the Multilayer Perceptron(MLP) using MXNet's symbolic interface
data = mx.sym.var('data')
# flatten the data from 4D shape into 2D
# (batch_size, num_channel, width, height) -> 
# (batch_size, num_channel*width*height)
data = mx.sym.flatten(data=data)
fc1 = mx.sym.FullyConnected(data=data, num_hidden=128)
act1 = mx.sym.Activation(data=fc1, act_type='relu')
fc2 = mx.sym.FullyConnected(data=act1, num_hidden=64)
act2 = mx.sym.Activation(data=fc2, act_type='relu')
# MNIST has 10 classes
fc3 = mx.sym.FullyConnected(data=act2, num_hidden=10)
# Softmax with cross entropy loss
mlp = mx.sym.SoftmaxOutput(data=fc3, name='softmax')

In [19]:
mlt_model = mx.mod.Module(symbol=mlp, context=ctx)
mlt_model.fit(train_iter,
              eval_data=val_iter,
              optimizer='sgd',
              optimizer_params={'learning_rate':0.01},
              eval_metric='acc',
              batch_end_callback=mx.callback.Speedometer(batch_size, 50),
              num_epoch=50)

INFO:root:Epoch[0] Batch [0-50]	Speed: 106827.50 samples/sec	accuracy=0.467451
INFO:root:Epoch[0] Batch [50-100]	Speed: 126394.61 samples/sec	accuracy=0.786800
INFO:root:Epoch[0] Batch [100-150]	Speed: 127822.90 samples/sec	accuracy=0.870000
INFO:root:Epoch[0] Batch [150-200]	Speed: 125753.42 samples/sec	accuracy=0.892400
INFO:root:Epoch[0] Batch [200-250]	Speed: 127343.23 samples/sec	accuracy=0.916000
INFO:root:Epoch[0] Batch [250-300]	Speed: 125015.77 samples/sec	accuracy=0.931800
INFO:root:Epoch[0] Train-accuracy=0.821994
INFO:root:Epoch[0] Time cost=0.280
INFO:root:Epoch[0] Validation-accuracy=0.942738
INFO:root:Epoch[1] Batch [0-50]	Speed: 120883.77 samples/sec	accuracy=0.937843
INFO:root:Epoch[1] Batch [50-100]	Speed: 94224.80 samples/sec	accuracy=0.946000
INFO:root:Epoch[1] Batch [100-150]	Speed: 80579.42 samples/sec	accuracy=0.956600
INFO:root:Epoch[1] Batch [150-200]	Speed: 131254.12 samples/sec	accuracy=0.948200
INFO:root:Epoch[1] Batch [200-250]	Speed: 167331.74 samples/sec	

INFO:root:Epoch[13] Batch [150-200]	Speed: 113578.74 samples/sec	accuracy=0.999000
INFO:root:Epoch[13] Batch [200-250]	Speed: 135054.42 samples/sec	accuracy=0.998400
INFO:root:Epoch[13] Batch [250-300]	Speed: 140907.33 samples/sec	accuracy=0.999800
INFO:root:Epoch[13] Train-accuracy=0.999137
INFO:root:Epoch[13] Time cost=0.325
INFO:root:Epoch[13] Validation-accuracy=0.975952
INFO:root:Epoch[14] Batch [0-50]	Speed: 75570.05 samples/sec	accuracy=0.999804
INFO:root:Epoch[14] Batch [50-100]	Speed: 67891.19 samples/sec	accuracy=1.000000
INFO:root:Epoch[14] Batch [100-150]	Speed: 88243.18 samples/sec	accuracy=1.000000
INFO:root:Epoch[14] Batch [150-200]	Speed: 145065.37 samples/sec	accuracy=1.000000
INFO:root:Epoch[14] Batch [200-250]	Speed: 144880.97 samples/sec	accuracy=0.998800
INFO:root:Epoch[14] Batch [250-300]	Speed: 145727.65 samples/sec	accuracy=0.999200
INFO:root:Epoch[14] Train-accuracy=0.999613
INFO:root:Epoch[14] Time cost=0.337
INFO:root:Epoch[14] Validation-accuracy=0.977619
IN

INFO:root:Epoch[26] Batch [250-300]	Speed: 127953.92 samples/sec	accuracy=1.000000
INFO:root:Epoch[26] Train-accuracy=1.000000
INFO:root:Epoch[26] Time cost=0.310
INFO:root:Epoch[26] Validation-accuracy=0.979405
INFO:root:Epoch[27] Batch [0-50]	Speed: 124622.77 samples/sec	accuracy=1.000000
INFO:root:Epoch[27] Batch [50-100]	Speed: 93549.77 samples/sec	accuracy=1.000000
INFO:root:Epoch[27] Batch [100-150]	Speed: 94036.37 samples/sec	accuracy=1.000000
INFO:root:Epoch[27] Batch [150-200]	Speed: 105192.64 samples/sec	accuracy=1.000000
INFO:root:Epoch[27] Batch [200-250]	Speed: 130085.79 samples/sec	accuracy=1.000000
INFO:root:Epoch[27] Batch [250-300]	Speed: 128343.10 samples/sec	accuracy=1.000000
INFO:root:Epoch[27] Train-accuracy=1.000000
INFO:root:Epoch[27] Time cost=0.313
INFO:root:Epoch[27] Validation-accuracy=0.978810
INFO:root:Epoch[28] Batch [0-50]	Speed: 126360.34 samples/sec	accuracy=1.000000
INFO:root:Epoch[28] Batch [50-100]	Speed: 93577.74 samples/sec	accuracy=1.000000
INFO:r

INFO:root:Epoch[39] Time cost=0.295
INFO:root:Epoch[39] Validation-accuracy=0.978452
INFO:root:Epoch[40] Batch [0-50]	Speed: 123645.54 samples/sec	accuracy=1.000000
INFO:root:Epoch[40] Batch [50-100]	Speed: 93010.81 samples/sec	accuracy=1.000000
INFO:root:Epoch[40] Batch [100-150]	Speed: 93507.23 samples/sec	accuracy=1.000000
INFO:root:Epoch[40] Batch [150-200]	Speed: 106232.78 samples/sec	accuracy=1.000000
INFO:root:Epoch[40] Batch [200-250]	Speed: 128046.11 samples/sec	accuracy=1.000000
INFO:root:Epoch[40] Batch [250-300]	Speed: 125593.76 samples/sec	accuracy=1.000000
INFO:root:Epoch[40] Train-accuracy=1.000000
INFO:root:Epoch[40] Time cost=0.317
INFO:root:Epoch[40] Validation-accuracy=0.978810
INFO:root:Epoch[41] Batch [0-50]	Speed: 123776.90 samples/sec	accuracy=1.000000
INFO:root:Epoch[41] Batch [50-100]	Speed: 93507.23 samples/sec	accuracy=1.000000
INFO:root:Epoch[41] Batch [100-150]	Speed: 101555.03 samples/sec	accuracy=1.000000
INFO:root:Epoch[41] Batch [150-200]	Speed: 102884.

In [22]:
# Prediction
test_iter = mx.io.NDArrayIter(X_test_final, None, batch_size)
prob = mlt_model.predict(test_iter)
print(prob)
assert prob.shape == (len(X_test_final), 10)

y = []
for p in list(prob):
    y.append(list(p).index(np.max(p)))


[[2.6356227e-16 1.3847826e-21 1.0000000e+00 ... 9.0283124e-18
  4.9020990e-17 8.3351170e-19]
 [1.0000000e+00 3.1273072e-20 2.4359449e-12 ... 6.5233319e-15
  2.6917599e-17 1.4395826e-15]
 [1.4925286e-12 1.8818085e-09 1.2652862e-10 ... 2.0895181e-09
  5.0856875e-07 9.9999189e-01]
 ...
 [3.9749007e-25 1.2200634e-21 3.0529557e-18 ... 3.1961244e-20
  2.8325363e-14 1.6831983e-13]
 [1.2695766e-14 9.2866932e-17 1.7610825e-15 ... 7.0017975e-10
  1.0404935e-11 9.9999928e-01]
 [4.0441237e-18 6.6697186e-18 1.0000000e+00 ... 2.3918278e-17
  4.2069406e-14 1.0260664e-18]]
<NDArray 28000x10 @cpu(0)>


In [24]:
# val test dataset
test_iter = mx.io.NDArrayIter(X_test, y_test, batch_size)
# predict accuracy of mlp
acc = mx.metric.Accuracy()
mlt_model.score(test_iter, acc)
print(acc)

EvalMetric: {'accuracy': 0.9783333333333334}


In [25]:
df = pd.DataFrame({'ImageId': [x for x in range(1, len(y) + 1)], 'label': y})
df.to_csv('submission.csv', index=False)

In [39]:
# import operator
from numpy import *
def toInt(array):
    array = mat(array)
    m,n=shape(array)
    new=zeros((m,n))
    for i in range(m):
        for j in range(n):
            new[i,j] = int(array[i,j])
    return new

In [40]:
def nomalizing(array):
    m,n = shape(array)
    for i in range(m):
        for j in range(n):
            if array[i,j] !=0 :
                array[i,j] = 1
    return array

In [49]:
def showDataIndexK(k):
    print(nomalizing(toInt(X_test_final[k])).reshape(28,28)[:,4:-4])
    print("predict value: ", y[k])
    
showDataIndexK(120)

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 0.